Platform

In [2]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
SS_VER = 'SS_VER_2_1'


Setup - .....

In [4]:
#%run "../../Setup_Scripts/Setup_Generic.ipynb"

Firmware bauen

In [ ]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../../hardware/victims/firmware/basic-passwdcheck
make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 -j

Auf gerät übertragen

In [9]:
#cw.program_target(scope, prog, "../../../hardware/victims/firmware/basic-passwdcheck/basic-passwdcheck-{}.hex".format(PLATFORM))

Matplot

In [14]:
%matplotlib notebook
import matplotlib.pylab as plt
plt.plot(range(0, 10))

<IPython.core.display.Javascript object>

In [ ]:
plt.plot(trace_hrgth label = "line 1") [0:1000]
plt.plot(trace_h label = "line 2") [0:1000]
plt.legend()
plt.show()

Capture Trace standi

In [ ]:
def capture_trace(_ignored=None):
    ktp = cw.ktp.Basic()
    
    key, text = ktp.next()
    return cw.capture_trace(scope, target, text).wave

Capture Trace pw knacken

In [ ]:
def cap_pass_trace(pass_guess):
    reset_target(scope)
    num_char = target.in_waiting()
    while num_char > 0:
        target.read(num_char, 10)
        time.sleep(0.01)
        num_char = target.in_waiting()

    scope.arm()
    target.write(pass_guess)
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    trace = scope.get_last_trace()
    return trace

In [ ]:
wave_test = capture_trace()
cw.plot(wave_test)

Error handling

In [11]:
raise NotImplementedError("Add your code here, and delete this.")

NotImplementedError: Add your code here, and delete this.

for each

In [ ]:
for x in range(2):

In [3]:
def mean(X):
    return np.sum(X, axis=0)/len(X)

def std_dev(X, X_bar):
    return np.sqrt(np.sum((X-X_bar)**2, axis=0))

def cov(X, X_bar, Y, Y_bar):
    return np.sum((X-X_bar)*(Y-Y_bar), axis=0)

In [8]:
import numpy as np

t_bar = np.sum(trace_array, axis=0)/len(trace_array)
o_t = np.sqrt(np.sum((trace_array - t_bar)**2, axis=0))

cparefs = [0] * 16 #put your key byte guess correlations here
bestguess = [0] * 16 #put your key byte guesses here

for bnum in trange(0, 16):
    maxcpa = [0] * 256
    for kguess in range(0, 256):

        hws = np.array([[HW[aes_internal(textin[bnum],kguess)] for textin in textin_array]]).transpose()
        hws_bar = mean(hws)
        o_hws = std_dev(hws, hws_bar)
        correlation = cov(trace_array, t_bar, hws, hws_bar)
        cpaoutput = correlation/(o_t*o_hws)
        maxcpa[kguess] = max(abs(cpaoutput))
    bestguess[bnum] = np.argmax(maxcpa)
    cparefs[bnum] = max(maxcpa)


print("Best Key Guess: ", end="")
for b in bestguess: print("%02x " % b, end="")
print("\n", cparefs)

NameError: name 'trace_array' is not defined

In [9]:
import inspect


inspect.getsource(np.argmax
        )

'@array_function_dispatch(_argmax_dispatcher)\ndef argmax(a, axis=None, out=None):\n    """\n    Returns the indices of the maximum values along an axis.\n\n    Parameters\n    ----------\n    a : array_like\n        Input array.\n    axis : int, optional\n        By default, the index is into the flattened array, otherwise\n        along the specified axis.\n    out : array, optional\n        If provided, the result will be inserted into this array. It should\n        be of the appropriate shape and dtype.\n\n    Returns\n    -------\n    index_array : ndarray of ints\n        Array of indices into the array. It has the same shape as `a.shape`\n        with the dimension along `axis` removed.\n\n    See Also\n    --------\n    ndarray.argmax, argmin\n    amax : The maximum value along a given axis.\n    unravel_index : Convert a flat index into an index tuple.\n    take_along_axis : Apply ``np.expand_dims(index_array, axis)``\n                      from argmax to an array as if by cal

In [ ]:
__kernel void aes_internal(__global const uchar *inputdata,
                            __global const uchar *key,
                            __global uchar *result) {

    int idx = get_global_id(0);

    result[idx] = sbox[inputdata[idx] ^ key[idx]];
}

In [13]:
import numpy as np
import pyopencl as cl

# Define kernel code
kernel_code = """
__kernel void square(__global const float *a, __global float *c)
{
    int gid = get_global_id(0);
    c[gid] = 1;
}
"""

# Initialize OpenCL context, command queue, and device
platforms = cl.get_platforms()
devices = [device for platform in platforms for device in platform.get_devices()]
ctx = cl.Context(devices)
queue = cl.CommandQueue(ctx, devices[0])

# Create input and output numpy arrays
a = np.array([1, 2, 3, 4, 5], dtype=np.float32)
c = np.empty_like(a)

# Create input and output OpenCL buffers
mf = cl.mem_flags
a_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a)
c_buf = cl.Buffer(ctx, mf.WRITE_ONLY, c.nbytes)

# Compile kernel and execute on the device
prg = cl.Program(ctx, kernel_code).build()
prg.square(queue, a.shape, None, a_buf, c_buf)

# Copy results from OpenCL buffer to numpy array
cl.enqueue_copy(queue, c, c_buf)
print(devices)
print(a.dtype)
print(c)

[<pyopencl.Device 'NVIDIA GeForce RTX 2070 SUPER' on 'NVIDIA CUDA' at 0x1e793edc680>]
float32
[1. 1. 1. 1. 1.]


In [ ]:
def mean(X):
    return np.sum(X, axis=0)/len(X)

def std_dev(X, X_bar):
    return np.sqrt(np.sum((X-X_bar)**2, axis=0))

def cov(X, X_bar, Y, Y_bar):
    return np.sum((X-X_bar)*(Y-Y_bar), axis=0)
sbox = [
    # 0    
    0x63, ...
   
]

def aes_internal(inputdata, key):
    return sbox[inputdata ^ key]

HW = [bin(n).count("1") for n in range(0, 256)]


for bnum in trange(0, 16):

    for kguess in range(0, 256):

        hws = np.array([[HW[aes_internal(textin[bnum],kguess)] for textin in textin_array]]).transpose()
        hws_bar = mean(hws)

end

In [ ]:
scope.dis()
target.dis()

In [9]:
import numpy as np
import pyopencl as cl

# Initialisieren Sie das OpenCL-Setup
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)

# Definieren Sie den Kernel
kernel = """
__kernel void my_kernel(__global float* output) {
    int i = get_global_id(0);
    output[i] = sin((float)i);
    printf("Sinus von %d: %f", i, output[i]);
}
"""

# Kompilieren Sie den Kernel
prg = cl.Program(ctx, kernel).build()

# Erstellen Sie den Eingangs- und Ausgangsbuffer
n = 100
output = np.zeros(n, dtype=np.float32)
output_buf = cl.Buffer(ctx, cl.mem_flags.WRITE_ONLY, output.nbytes)

# Führen Sie den Kernel aus
prg.my_kernel(queue, (n,), None, output_buf)
cl.enqueue_copy(queue, output, output_buf)

# Geben Sie die Ausgabe aus
print(output)

[ 0.          0.841471    0.90929747  0.14112    -0.7568025  -0.9589243
 -0.2794155   0.6569866   0.98935825  0.4121185  -0.54402107 -0.9999902
 -0.53657293  0.42016706  0.9906074   0.65028787 -0.28790334 -0.96139747
 -0.75098723  0.1498772   0.9129453   0.8366556  -0.00885131 -0.84622043
 -0.9055784  -0.13235176  0.76255846  0.95637596  0.2709058  -0.6636339
 -0.9880316  -0.40403765  0.5514267   0.99991184  0.5290827  -0.42818266
 -0.99177885 -0.6435381   0.29636857  0.96379536  0.7451132  -0.15862267
 -0.91652155 -0.8317748   0.01770193  0.8509035   0.90178835  0.12357312
 -0.7682547  -0.95375264 -0.26237485  0.6702292   0.9866276   0.39592513
 -0.5587891  -0.99975514 -0.521551    0.43616477  0.99287266  0.636738
 -0.30481064 -0.9661178  -0.7391807   0.1673557   0.92002606  0.8268287
 -0.02655116 -0.85552    -0.8979277  -0.11478481  0.77389073  0.95105463
  0.25382337 -0.676772   -0.9851463  -0.38778165  0.56610763  0.9995202
  0.5139784  -0.44411266 -0.9938887  -0.629888    0.313228

In [ ]:
 // Compute t_bar
    for (int i = 0; i < n_traces; i++) {
        t_bar += trace_array[i * 5000 + bnum];
    }
    t_bar /= n_traces;

    // Compute o_t
    for (int i = 0; i < n_traces; i++) {
        o_t += pow(trace_array[i * 5000 + bnum] - t_bar, 2);
    }
    o_t = sqrt(o_t);

In [ ]:
printf("\t");
        float cpaoutput = fabs( ((float)o_t * o_hws));

        if (cpaoutput > maxcpa) {
            maxcpa = fabs(cpaoutput);
            key_byte_guess = kguess;
        }
        // Speichern Sie die Debugging-Werte in den zusätzlichen Arrays
        if (kguess == 255) {
            debug_t_bar[bnum] = hws_bar;
            debug_o_hws[bnum] = o_hws;
            debug_o_t[bnum] = (float)o_t;
            // debug_correlation[bnum] = correlation;
        }
    }

    bestguess[bnum] = key_byte_guess;
    cparefs[bnum] = maxcpa;
    //printf("best guess for byte %d: %d, correlation: %f\t", bnum, bestguess[bnum], cparefs[bnum]);

In [ ]:
correlation_array[j] = sum;
}
        // printf("hws[0]: %f, hws_diff: %f, hws_bar: %f\t", hws[50], hws_diff, hws_bar);

        // Drucke die Werte in correlation_array
        
        //printf("Correlation values: ");
        printf("kguess: %d, correlation_array[0]: %f\t", kguess, correlation_array[0]);
        
        
        
        
}

In [ ]:
        double max_corr = 0.0;
        for (int i = 0; i < 5000; i++) {
            if (fabs(correlation_array[i]) > max_corr) {
                max_corr = fabs(correlation_array[i]);
            }
        }

        if (max_corr > maxcpa) {
            maxcpa = max_corr;
            key_byte_guess = kguess;
        }
    }
    
    bestguess[bnum] = key_byte_guess;
    cparefs[bnum] = maxcpa;
}

In [10]:
import numpy as np

# Laden der .npy Datei
trace_array = np.load("lab4_2_traces.npy")

# Speichern der Daten in eine .txt Datei
np.savetxt("trace_array.txt", trace_array)

In [11]:
with open('trace_array.txt', 'r') as in_file, open('output.txt', 'w') as out_file:
    for line in in_file:
        line = line.rstrip('\n')
        numbers = line.split()
        # Schreiben Sie die Originalzeile in die Ausgabedatei
        out_file.write(line)
        # Fügen Sie die duplizierten Zahlen am Ende der Zeile hinzu
        out_file.write(' ' + ' '.join(numbers) + '\n')

In [ ]:
result = np.array([0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6, 0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c])

# Compare the output of best_bestguess and bestguess
if np.array_equal(best_bestguess, result):
    print("\u2714 The values of best_bestguess and bestguess are the same.") 
else:
    print("\u2718 The values of best_bestguess and bestguess are different.")  


In [13]:
import os
# Initialisieren Sie die Eingabe als die ursprüngliche Datei
input_file = 'trace_array.txt'

for i in range(10):
    # Erstellen Sie eine temporäre Ausgabedatei für diese Iteration
    output_file = f'temp_{i}.txt'

    with open(input_file, 'r') as in_file, open(output_file, 'w') as out_file:
        for line in in_file:
            line = line.rstrip('\n')
            numbers = line.split()
            # Schreiben Sie die Originalzeile in die Ausgabedatei
            out_file.write(line)
            # Fügen Sie die duplizierten Zahlen am Ende der Zeile hinzu
            out_file.write(' ' + ' '.join(numbers) + '\n')

    # Setzen Sie die Eingabe auf die Ausgabe dieser Iteration für die nächste Iteration
    input_file = output_file

# Benennen Sie die letzte Ausgabedatei um
os.rename(output_file, 'output_new.txt')

FileExistsError: [WinError 183] Eine Datei kann nicht erstellt werden, wenn sie bereits vorhanden ist: 'temp_9.txt' -> 'output.txt'

In [5]:
import numpy as np
np.load('textin_array_test.npy')

array([[131, 170,  38, ..., 156,   4, 190],
       [ 54,  82, 128, ..., 247,  85, 152],
       [234, 237,  14, ..., 127,  33,  64],
       ...,
       [150, 143,  83, ...,  40,  33, 232],
       [ 37, 112,  59, ...,  21,  89,  70],
       [224,   0,  28, ...,  88, 254,  99]], dtype=uint8)

In [15]:
import numpy as np

# Laden Sie das Numpy-Array aus der Datei
data = np.load('trace_array_9_100.npy')

# Kopieren und am Ende des Arrays anhängen
data_extended = np.concatenate((data, data), axis=0)

# Speichern Sie das erweiterte Numpy-Array in einer Datei
np.save('trace_array_9_200.npy', data_extended)